In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from scrapy.selector import Selector

import re, requests, csv
import lxml

# Regex
regex_url = r"(http(s?):\/\/(www)?[A-Za-z0-9-?\/\.]+)"
regex_hastags = r"(#[A-Za-z0-9]{2,})"

# File url
file_url = '../datasets/foxnewshealth.txt'

# Basic list
csv_header = ['tweetid','date','title']
frame_header = ['tweetid','date','title','what']

# Support scripts
%run -i scripts.py

In [2]:
frame = pd.read_table(file_url, sep='|', header= None, names = csv_header, encoding="utf-8")
frame.head()

,tweetid,date,title
0,585942799561928704,Wed Apr 08 23:10:24 +0000 2015,Injury prevention programs unpopular with high...
1,585934004089683969,Wed Apr 08 22:35:27 +0000 2015,6 dietary changes to make midlife http://ow.ly...
2,585926494054195201,Wed Apr 08 22:05:36 +0000 2015,Massachusetts governor gets head shaved to sup...
3,585921515922788354,Wed Apr 08 21:45:49 +0000 2015,Dad wins 3 marathons in 8 days; winnings to he...
4,585915150449934338,Wed Apr 08 21:20:32 +0000 2015,Possible cure for melanoma? http://ow.ly/LlLg8


In [3]:
frame.shape

(2000, 3)

In [4]:
frame.loc[:,'title'][0]

'Injury prevention programs unpopular with high school coaches http://ow.ly/Lma9z'

In [5]:
# Split Url & Text
frame['url'] = frame['title'].map(lambda row: get_url(regex_url, row))
frame['title'] = frame['title'].map(lambda row: split_txt_form_url(regex_url, row))
frame.head()

http://ow.ly/Lma9z
http://ow.ly/LlYfl
http://ow.ly/LlLI3
http://ow.ly/LlLyZ
http://ow.ly/LlLg8
http://ow.ly/LlL69
http://ow.ly/LlKYU
http://ow.ly/LlmOO
http://ow.ly/LlmKx
http://ow.ly/Llmaj
http://ow.ly/LllIf
http://ow.ly/LllF0
http://ow.ly/LllBM
http://ow.ly/Lllze
http://ow.ly/LllvX
http://ow.ly/LlRmW
http://ow.ly/LllsI
http://ow.ly/LllpI
http://ow.ly/LlllW
http://ow.ly/LllhY
http://ow.ly/Lllah
http://ow.ly/LlkWI
http://ow.ly/Llc4n
http://ow.ly/Ljqd1
http://ow.ly/LjeGL
http://ow.ly/LiUWe
http://ow.ly/LiUSJ
http://ow.ly/LiUQ7
http://ow.ly/LiUMG
http://ow.ly/LiCJS
http://ow.ly/LiCDd
http://ow.ly/LiCwz
http://ow.ly/LiCs4
http://ow.ly/LiCmF
http://ow.ly/LiCiC
http://ow.ly/LiCeE
http://ow.ly/LigQS
http://ow.ly/LigiQ
http://ow.ly/Lig2W
http://ow.ly/LidGT
http://ow.ly/LidDJ
http://ow.ly/LidzD
http://ow.ly/LiduS
http://ow.ly/Lidmt
http://ow.ly/LicX5
http://ow.ly/LicMo
http://ow.ly/LicJy
http://ow.ly/Lghqn
http://ow.ly/Lghiy
http://ow.ly/LghdN
http://ow.ly/LfQ4x
http://ow.ly/LfNTK
http://ow.ly

http://ow.ly/GSLeo
http://ow.ly/GSAxu
http://ow.ly/GSAo5
http://ow.ly/GSA83
http://ow.ly/GSzXI
http://ow.ly/GSzvY
http://ow.ly/GSzmy
http://pbs.twimg.com/media/B6sIaX7CAAAReD0.jpg
http://ow.ly/GSiUs
http://ow.ly/GSiKu
http://ow.ly/GSiCR
http://ow.ly/GSitD
http://ow.ly/GSijs
http://ow.ly/GSi8h
http://ow.ly/GSi1o
http://ow.ly/GShOX
http://ow.ly/GShIM
http://ow.ly/GSfZv
http://ow.ly/GSfRo
http://ow.ly/GSfFK
http://ow.ly/GSeQF
http://ow.ly/GPJbP
http://ow.ly/GPJ5n
http://ow.ly/GPILn
http://ow.ly/GPrhu
http://ow.ly/GPkR8
http://ow.ly/GOM7y
http://ow.ly/GOJN3
http://ow.ly/GOJEV
http://ow.ly/GOHkA
http://ow.ly/GODcL
http://ow.ly/GOCYS
http://ow.ly/GOCKO
http://ow.ly/GOCCX
http://ow.ly/GOukr
http://ow.ly/GOubw
http://ow.ly/GOtBe
http://ow.ly/GOtp2
http://ow.ly/GOtgr
http://ow.ly/GOt6a
http://ow.ly/GOsKx
http://ow.ly/GOsCu
http://ow.ly/GIrUk
http://ow.ly/GIkzT
http://ow.ly/GI7AM
http://ow.ly/GI7wI
http://ow.ly/GHY7B
http://ow.ly/GHXZA
http://ow.ly/GHXU4
http://ow.ly/GHMIG
http://ow.ly/GHLyJ
htt

http://ow.ly/ICHNb
http://ow.ly/ICx9B
http://ow.ly/ICx0N
http://ow.ly/ICvOm
http://ow.ly/ICvVR
http://ow.ly/ICvC1
http://ow.ly/ICvtf
http://ow.ly/ICKK9
http://ow.ly/ICw4B
http://ow.ly/ICBfu
http://ow.ly/ICvhw
http://ow.ly/ICkIv
http://ow.ly/ICkCb
http://ow.ly/Izc5P
http://ow.ly/IzbVF
http://ow.ly/Iz0fK
http://ow.ly/IyVm8
http://ow.ly/IyVbt
http://ow.ly/IyV8X
http://ow.ly/IyAU9
http://ow.ly/IyAMc
http://ow.ly/IyAF7
http://ow.ly/IyA8o
http://ow.ly/IyzZh
http://ow.ly/IyzyG
http://ow.ly/Iyzqa
http://ow.ly/Iyzjv
http://ow.ly/Iyzd5
http://ow.ly/Iyz4w
http://ow.ly/IyyYd
http://ow.ly/IyyFM
http://ow.ly/IyytU
http://ow.ly/Iyyla
http://ow.ly/Iyy2M
http://ow.ly/IyxXq
http://ow.ly/IyxQd
http://ow.ly/IyxeO
http://ow.ly/Iyx7Y
http://ow.ly/IvBEH
http://ow.ly/IuSam
http://ow.ly/IuRUA
http://ow.ly/IumXa
http://ow.ly/Iub0O
http://ow.ly/IuaPJ
http://ow.ly/Iu8XI
http://ow.ly/Iu8R2
http://ow.ly/Iu8Hd
http://ow.ly/Iu8xW
http://ow.ly/Iu8nU
http://ow.ly/Iu8f4
http://ow.ly/Iu87l
http://ow.ly/Iu80H
http://ow.ly

,tweetid,date,title,url
0,585942799561928704,Wed Apr 08 23:10:24 +0000 2015,Injury prevention programs unpopular with high...,http://ow.ly/Lma9z
1,585934004089683969,Wed Apr 08 22:35:27 +0000 2015,6 dietary changes to make midlife,http://ow.ly/LlYfl
2,585926494054195201,Wed Apr 08 22:05:36 +0000 2015,Massachusetts governor gets head shaved to sup...,http://ow.ly/LlLI3
3,585921515922788354,Wed Apr 08 21:45:49 +0000 2015,Dad wins 3 marathons in 8 days; winnings to he...,http://ow.ly/LlLyZ
4,585915150449934338,Wed Apr 08 21:20:32 +0000 2015,Possible cure for melanoma?,http://ow.ly/LlLg8


In [6]:
frame_urls = frame['url'].values

In [ ]:
# Run for this jupyter notebook
title_attempt = True
for key,url in enumerate(frame_urls):

    #     url = 'http://bit.ly/VymaIc'

    # Start the connection
    print(url)
    res = requests.get(url)
    
    # Check status header
    if res.status_code == 200:
        print("Header: ", 200)

        # Initialize BeautifulSoup()
        soup = BeautifulSoup(res.content, 'lxml')

        # Get return content as text
        # text_html = res.text
        
        # Add columns to dataframe
        if title_attempt:
            df_temp = pd.DataFrame(columns=['author','content','tags'])
            frame = pd.concat([frame,df_temp])
            title_attempt = False
        
        ahref = soup.find('a',{'class':'article_link'})
        if ahref:
            author = ahref.text            
#             frame['author'][key] = author[:author.index(",")].lstrip()
            frame['author'][key] = author.lstrip()
        else:
            frame['author'] = ''
        frame['content'][key] = soup.find('div',{'class':'article-body'})
        frame['tags'][key] = get_hastags_out(regex_hastags,frame['title'][key])
        frame['title'][key] = stript_hastags(frame['title'][key])
    else:
        print("[ERROR]: ", res.status_code)

http://ow.ly/Lma9z
[ERROR]:  404
http://ow.ly/LlYfl
[ERROR]:  404
http://ow.ly/LlLI3
[ERROR]:  404
http://ow.ly/LlLyZ
[ERROR]:  404
http://ow.ly/LlLg8
[ERROR]:  404
http://ow.ly/LlL69
[ERROR]:  404
http://ow.ly/LlKYU
[ERROR]:  404
http://ow.ly/LlmOO
[ERROR]:  404
http://ow.ly/LlmKx
[ERROR]:  404
http://ow.ly/Llmaj
[ERROR]:  404
http://ow.ly/LllIf
[ERROR]:  404
http://ow.ly/LllF0
[ERROR]:  404
http://ow.ly/LllBM
[ERROR]:  404
http://ow.ly/Lllze
[ERROR]:  404
http://ow.ly/LllvX
[ERROR]:  404
http://ow.ly/LlRmW
[ERROR]:  404
http://ow.ly/LllsI
[ERROR]:  404
http://ow.ly/LllpI
[ERROR]:  404
http://ow.ly/LlllW
[ERROR]:  404
http://ow.ly/LllhY
[ERROR]:  404
http://ow.ly/Lllah
[ERROR]:  404
http://ow.ly/LlkWI
[ERROR]:  404
http://ow.ly/Llc4n
[ERROR]:  404
http://ow.ly/Ljqd1
[ERROR]:  404
http://ow.ly/LjeGL
[ERROR]:  404
http://ow.ly/LiUWe
[ERROR]:  404
http://ow.ly/LiUSJ
[ERROR]:  404
http://ow.ly/LiUQ7
[ERROR]:  404
http://ow.ly/LiUMG
[ERROR]:  404
http://ow.ly/LiCJS
[ERROR]:  404
http://ow.

scripts.py:24: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  
scripts.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
scripts.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Strip "#" within string
scripts.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  def s

http://ow.ly/KUOIc
[ERROR]:  404
http://ow.ly/KTxKM
[ERROR]:  404
http://ow.ly/KTqiA
[ERROR]:  404
http://ow.ly/KTjYF
[ERROR]:  404
http://ow.ly/KT4rX
[ERROR]:  404
http://ow.ly/KT4lJ
[ERROR]:  404
http://ow.ly/KT2Rs
[ERROR]:  404
http://ow.ly/KSVyW
[ERROR]:  404
http://ow.ly/KSUj0
[ERROR]:  404
http://ow.ly/KSSRi
[ERROR]:  404
http://ow.ly/KSyFR
[ERROR]:  404
http://ow.ly/KSyzY
[ERROR]:  404
http://ow.ly/KSyuc
[ERROR]:  404
http://ow.ly/KSymM
[ERROR]:  404
http://ow.ly/KSyim
[ERROR]:  404
http://ow.ly/KSycI
[ERROR]:  404
http://ow.ly/KSPBf
[ERROR]:  404
http://ow.ly/KSy9A
[ERROR]:  404
http://ow.ly/KSy5o
[ERROR]:  404
http://ow.ly/KSy2a
[ERROR]:  404
http://ow.ly/KSxZV
[ERROR]:  404
http://ow.ly/KQAa5
[ERROR]:  404
http://ow.ly/KQlxb
[ERROR]:  404
http://ow.ly/KQ7H1
[ERROR]:  404
http://ow.ly/KQ5KO
[ERROR]:  404
http://ow.ly/KQ5yN
[ERROR]:  404
http://ow.ly/KPDVA
[ERROR]:  404
http://ow.ly/KPDSP
[ERROR]:  404
http://ow.ly/KPDQ2
[ERROR]:  404
http://ow.ly/KPDLK
[ERROR]:  404
http://ow.

In [ ]:
frame.head()

In [ ]:
frame.shape